In [1]:
from pyspark.sql import SparkSession

minio_endpoint = "http://minio:9000"
minio_access_key = "admin"
minio_secret_key = "password"
minio_bucket = "gold"

spark = SparkSession.builder \
    .appName("sometest") \
    .master("spark://spark-master:7077") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "/opt/spark-events") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

print("✅ Spark initialized successfully!")
print("Spark version:", spark.version)
print("Master URL:", spark.sparkContext.master)


/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 01:01:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark initialized successfully!
Spark version: 3.4.0
Master URL: spark://spark-master:7077


In [2]:
df = spark.read.parquet("s3a://gold/sales/product_analytics")
df.show()


26/01/19 01:01:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
[Stage 1:>                                                          (0 + 1) / 1]

+------------+--------------------+----------------+----------------------+-----------------+------------+------------------+----------------------+--------------------+------------------------+------------+--------------------+
|product_name|    product_category|lifetime_revenue|lifetime_quantity_sold|avg_selling_price|avg_discount|total_transactions|total_customers_approx|processing_timestamp|lifetime_revenue_rounded|revenue_rank|market_share_percent|
+------------+--------------------+----------------+----------------------+-----------------+------------+------------------+----------------------+--------------------+------------------------+------------+--------------------+
|          TV|         Electronics|  1.0972787555E8|                  2258|         55702.66|       12.65|               739|                   739|2026-01-19 00:02:...|          1.0972787555E8|           1|               36.76|
|      CAMERA|         Photography|   9.188582725E7|                  1464|         

In [3]:
from pyspark.sql.functions import col
df = spark.read.parquet("s3a://gold/sales/product_analytics")
result_df = (
    df.select(
        col("product_name"),
        col("product_category"),
        col("lifetime_revenue_rounded").alias("revenue"),
        col("lifetime_quantity_sold").alias("quantity"),
        col("market_share_percent"),
        col("revenue_rank")
    )
    .orderBy(col("revenue_rank"))
    .limit(10)
)
result_df.show(truncate=False)

+------------+--------------------+--------------+--------+--------------------+------------+
|product_name|product_category    |revenue       |quantity|market_share_percent|revenue_rank|
+------------+--------------------+--------------+--------+--------------------+------------+
|TV          |Electronics         |1.0972787555E8|2258    |36.76               |1           |
|CAMERA      |Photography         |9.188582725E7 |1464    |30.78               |2           |
|LAPTOP      |Computing           |4.428780468E7 |748     |14.84               |3           |
|MOBILE      |Mobile & Accessories|1.737430245E7 |617     |5.82                |4           |
|WATCH       |Wearables           |1.061708988E7 |664     |3.56                |5           |
|HARDISK     |Storage             |6969519.4     |1090    |2.33                |6           |
|SPEAKER     |Audio               |6345423.19    |906     |2.13                |7           |
|MIC         |Audio               |4190334.79    |1057    |1

In [4]:
data = spark.range(0, 15)
data.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
+---+



In [5]:
df = spark.read.csv("s3a://test/taxi.csv")
df.show()
spark.stop()

+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+
|     _c0|                 _c1|                 _c2|            _c3|          _c4|       _c5|               _c6|         _c7|         _c8|         _c9|       _c10| _c11|   _c12|      _c13|        _c14|                _c15|        _c16|
+--------+--------------------+--------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_date...|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surch...|total_amount|
|       2|02/08/2018 05:07:...|02/08/2018 05:16:...|    